<a href="https://colab.research.google.com/github/OWEN-JUN/keras_/blob/master/day0802_colab_keras34_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.utils import np_utils
from keras.layers import *
from keras.callbacks import *

Using TensorFlow backend.


In [0]:
from keras.datasets import mnist
(x_train, y_train),(x_test,y_test) = mnist.load_data()

In [0]:

x_train, _, y_train, _ = train_test_split(x_train, y_train, random_state=66, test_size = 0.995)


x_train = x_train.reshape(x_train.shape[0], 28 ,28 ,1).astype('float32') / 255 # 6만행(무시) 나머지는 아래 input_shape값이 된다.
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255 # 0~1 사이로 수렴(minmax)시키기 위해 minmaxscaler같은거 필요없이 각 픽셀당 255의 값을 나누어서 아주 간단하게 데이터 전처리를 하는 과정
y_train = np_utils.to_categorical(y_train) # One Hot Incoding으로 데이터를 변환시킨다. 분류
y_test = np_utils.to_categorical(y_test)


In [0]:
data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range = 0.08,
    height_shift_range = 0.05,
    horizontal_flip=True)
    


In [0]:
model = Sequential()
#conv 신경망 설정

model.add(Conv2D(32,kernel_size=(3,3), input_shape=(28,28,1),activation="relu",padding="same"))
model.add(BatchNormalization())

# model.add(Dropout(0.3))
model.add(Conv2D(32,(4,4),activation="relu"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))



# model.add(MaxPooling2D(pool_size=4))
model.add(Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(64,(3,3),activation="relu"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))


model.add(Conv2D(200,(3,3),activation="relu",padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(200,(3,3),activation="relu",padding="same"))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=3))
model.add(Dropout(0.5))





model.add(Flatten())
# model.add(Dense(1500, activation="relu"))
# model.add(Dropout(0.3))

model.add(Dense(350, activation="relu"))
model.add(Dropout(0.3))

# model.add(Dense(30, activation="relu"))
# model.add(Dropout(0.5))
model.add(Dense(10,activation="softmax"))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [48]:
early_stoping_callback = EarlyStopping(monitor="val_loss",patience=30)

model.fit_generator(data_generator.flow(x_train, y_train, batch_size=10), 
                    # steps_per_epoch=len(x_train)//32,
                    steps_per_epoch=100,
                    epochs=300,
                    
                    validation_data=(x_test, y_test),
                    verbose=1,
                    callbacks=[early_stoping_callback]
                    )

Epoch 1/300
100/100 [==============================] - 10s 102ms/step - loss: 2.7034 - acc: 0.2290 - val_loss: 1.4463 - val_acc: 0.5200
Epoch 2/300
100/100 [==============================] - 8s 80ms/step - loss: 1.6448 - acc: 0.4490 - val_loss: 0.8873 - val_acc: 0.7145
Epoch 3/300
100/100 [==============================] - 8s 80ms/step - loss: 1.3263 - acc: 0.5530 - val_loss: 0.6157 - val_acc: 0.7884
Epoch 4/300
100/100 [==============================] - 8s 80ms/step - loss: 1.1285 - acc: 0.6160 - val_loss: 0.4574 - val_acc: 0.8488
Epoch 5/300
100/100 [==============================] - 8s 80ms/step - loss: 1.0149 - acc: 0.6500 - val_loss: 0.5728 - val_acc: 0.7863
Epoch 6/300
100/100 [==============================] - 8s 81ms/step - loss: 0.8888 - acc: 0.6950 - val_loss: 0.4045 - val_acc: 0.8503
Epoch 7/300
100/100 [==============================] - 8s 81ms/step - loss: 0.7417 - acc: 0.7570 - val_loss: 0.3541 - val_acc: 0.8776
Epoch 8/300
100/100 [==============================] - 8s 80

In [50]:
print("\n test acc: %.4f"%(model.evaluate(x_test, y_test)[1]))

10000/10000 [==============================] - 3s 257us/step

 test acc: 0.9576


In [34]:
data_generator.flow(x_train, y_train, batch_size=10)[0][0].shape

(10, 28, 28, 1)

In [35]:
len(data_generator.flow(x_train, y_train, batch_size=10))


30